# Avaliar Solução

In [2]:
from typing import List, Dict

def evaluate_solution(schedule: List[Dict]) -> int:
    conflicts = 0
    used_times = {}
    used_rooms = {}

    for class_info in schedule:
        for day_schedule in class_info['schedule']:
            for period in day_schedule['periods']:
                time_key = (day_schedule['day'], period.get('time'))
                room_key = (day_schedule['day'], period.get('time'), period.get('room'))

                # Conflito de horários
                if time_key in used_times:
                    conflicts += 1
                used_times[time_key] = True

                # Conflito de salas
                if room_key in used_rooms:
                    conflicts += 1
                used_rooms[room_key] = True

    return conflicts

# Avaliar Métricas

In [3]:
import json
import time

from typing import Any

class MetricsEvaluator:
    def __init__(self, model: str) -> None:
        self.model = model
        self.metrics = {
            "time_to_converge": 0,
            "best_conflicts": 0,
            "best_elite_fitness": 0,
            "best_chromosome": None,
            "iterations": 0,
            "avg_conflicts_history": [], # Pure fitness conflicts
            "avg_elite_fitness_history": [], # Fitness with conflicts resolved
            "time_to_evaluate": [],
        }

    def __repr__(self) -> str:
        return f"MetricsEvaluator({self.model})"

    def __str__(self) -> str:
        return f"MetricsEvaluator of Model: {self.model} with Metrics: {self.metrics}"

    def start_timer(self) -> None:
        self.start_time = time.time()

    def stop_timer(self):
        self.metrics["time_to_converge"] = (time.time() - self.start_time) / 60

    def start_iteration_timer(self) -> None:
        self.iteration_start_time = time.time()

    def stop_iteration_timer(self) -> None:
        self.metrics["time_to_evaluate"].append((time.time() - self.iteration_start_time) / 60)

    def update_metrics(self, iteration: int, avg_conflicts: float, avg_elite_fitness: float|None = None) -> None:
        self.metrics["iterations"] = iteration
        self.metrics["avg_conflicts_history"].append(avg_conflicts)
        if avg_elite_fitness is not None:
            self.metrics["avg_elite_fitness_history"].append(avg_elite_fitness)

    def update_best_metric(self, best_timetable: Any, best_conflicts: float, best_elite_fitness: float|None = None) -> None:
        self.metrics["best_timetable"] = best_timetable
        self.metrics["best_conflicts"] = best_conflicts
        if best_elite_fitness is not None:
            self.metrics["best_elite_fitness"] = best_elite_fitness

    def save_metrics(self, file_path: str) -> None:
        with open(file_path, "w") as file:
            json.dump(self.metrics, file, indent=4)


metrics_evaluator = MetricsEvaluator("Local Search Algorithm")

# Gerar Solução Inicial

In [4]:
from typing import List, Dict
import random

def generate_initial_solution(
    classes: List[Dict],
    teachers: List[Dict],
    rooms: List[Dict],
    times: List[Dict],
    subjects: List[Dict],
    days: List[Dict]
) -> List[Dict]:

    schedule = []
    for class_info in classes:
        class_schedule = []

        for day in days:
            day_name = day['name']
            day_schedule = {'day': day_name, 'periods': []}

            for time in times:
                time_info = f"{time['start']} - {time['end']}"
                available_rooms = [room for room in rooms if day['id'] in room['days'] and time['id'] in room['times']]

                suitable_rooms = [room for room in available_rooms if room['capacity'] >= class_info['studentsAmount']]
                random_room = random.choice(suitable_rooms) if suitable_rooms else None

                if random_room:
                    room_name = random_room['name']
                else:
                    room_name = "No suitable room available"

                random_teacher = random.choice(teachers)
                subject = random.choice(random_teacher['subjects'])
                teacher_name = random_teacher['name']

                period = {
                    "day": day_name,
                    "time": time_info,
                    "subject": next((sub['name'] for sub in subjects if sub['id'] == subject), None),
                    "teacher": teacher_name,
                    "room": room_name
                }
                day_schedule['periods'].append(period)

            class_schedule.append(day_schedule)

        schedule.append({"id": class_info['id'], "name": class_info['name'], "schedule": class_schedule})

    return schedule

# Local Search

In [5]:
import random
from copy import deepcopy
from typing import List, Dict

from copy import deepcopy
from typing import List, Dict
import random


def local_search(
    classes: List[Dict],
    teachers: List[Dict],
    rooms: List[Dict],
    times: List[Dict],
    subjects: List[Dict],
    days: List[Dict],
    max_iterations: int = 1000
) -> List[Dict]:

    def get_available_rooms(day_id: int, time_id: int) -> List[Dict]:
        return [room for room in rooms if day_id in room['days'] and time_id in room['times']]

    best_solution = generate_initial_solution(classes, teachers, rooms, times, subjects, days)
    best_score = evaluate_solution(best_solution)

    for _ in range(max_iterations):
        metrics_evaluator.start_iteration_timer()
        candidate_solution = deepcopy(best_solution)

        # Perturbação: Troca aleatória de alguns horários
        for class_info in candidate_solution:
            for day_schedule in class_info['schedule']:
                day_id = next(day['id'] for day in days if day['name'] == day_schedule['day'])
                for idx, period in enumerate(day_schedule['periods']):
                    available_rooms = get_available_rooms(day_id, idx + 1)
                    if available_rooms:
                        random_room = random.choice(available_rooms)
                        period['room'] = random_room['name']
                    else:
                        period['room'] = "No room available"

        candidate_score = evaluate_solution(candidate_solution)

        if candidate_score < best_score:
            best_solution = deepcopy(candidate_solution)
            best_score = candidate_score

            metrics_evaluator.update_metrics(best_solution, best_score)
            metrics_evaluator.stop_iteration_timer()

        metrics_evaluator.update_best_metric(candidate_solution, candidate_score)
        metrics_evaluator.stop_timer()

    return best_solution, metrics_evaluator

# Rodar o Algoritmo

In [ ]:
import json

if __name__ == "__main__":
    metrics_evaluator.start_timer()

    with open('./data/inputs/input1.json', 'r') as file:
        data = json.load(file)

    classes = data['classes']
    teachers = data['teachers']
    rooms = data['rooms']
    times = data['times']
    subjects = data['subjects']
    days = data['days']

    best_schedule, metrics_evaluator = local_search(classes, teachers, rooms, times, subjects, days)

    print(metrics_evaluator)

    with open('./data/outputs/timetabling_output.json', 'w') as file:
        json.dump(best_schedule, file, indent=2, ensure_ascii=False)

    print("Timetabling concluído. Resultados salvos em 'data/outputs/timetabling_output.json'.")